In [1]:
import pandas as pd

In [3]:
tt = pd.read_csv('C:/Users/t_louji/Downloads/train.csv')
tt.drop('Cabin',axis=1,inplace=True)

In [7]:
tt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(4)
memory usage: 76.6+ KB


In [57]:
tt['Title']=tt.Name.str.split(',').str[1].str.split('.').str[0]

In [40]:
tt.groupby('Title').Sex.value_counts().unstack()

Sex,female,male
Title,,
Capt,NaN,1.0
Col,NaN,2.0
Don,NaN,1.0
Dr,1.0,6.0
Jonkheer,NaN,1.0
Lady,1.0,NaN
Major,NaN,2.0
Master,NaN,40.0
Miss,182.0,NaN


In [20]:
tt.groupby('Title').Age.mean()

Title
 Capt            70.000000
 Col             58.000000
 Don             40.000000
 Dr              42.000000
 Jonkheer        38.000000
 Lady            48.000000
 Major           48.500000
 Master           4.574167
 Miss            21.773973
 Mlle            24.000000
 Mme             24.000000
 Mr              32.368090
 Mrs             35.898148
 Ms              28.000000
 Rev             43.166667
 Sir             49.000000
 the Countess    33.000000
Name: Age, dtype: float64

In [58]:
def chgn(s):
    if 'Lady' in s or 'Ms' in s or 'the Countess' in s:
        return ' Mrs'
    elif 'Mlle' in s or 'Mme' in s:
        return ' Miss'
    elif 'Major' in s or 'Don' in s or 'Rev' in s or 'Capt' in s or 'Jonkheer' in s or 'Col' in s  or 'Sir' in s:
        return ' Mr'
    else:
        return s

In [59]:
tt.Title=tt.Title.apply(chgn)

In [60]:
tt.groupby('Title').Sex.value_counts().unstack().index

Index([' Dr', ' Master', ' Miss', ' Mr', ' Mrs'], dtype='object', name='Title')

In [61]:
tt.groupby('Title').Sex.value_counts().unstack()

Sex,female,male
Title,,
Dr,1.0,6.0
Master,NaN,40.0
Miss,185.0,NaN
Mr,NaN,531.0
Mrs,128.0,NaN


In [62]:
tt.groupby('Title').Survived.value_counts().unstack()

Survived,0,1
Title,,
Dr,4,3
Master,17,23
Miss,55,130
Mr,447,84
Mrs,26,102


In [117]:
tt.drop('Name',axis=1,inplace=True)
tt.set_index('PassengerId')

In [66]:
tt.drop('Ticket',axis=1,inplace=True)


In [86]:
tt.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
Title          0
dtype: int64

In [70]:
tt.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [73]:
tt.Embarked=tt.Embarked.fillna('S')

In [74]:
tt.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Embarked         0
Title            0
dtype: int64

In [78]:
tt["Age"] = tt.groupby("Title").Age.transform(lambda x: x.fillna(x.mean()))


In [79]:
tt.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
Title          0
dtype: int64

In [89]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

x=pd.get_dummies(tt.drop('Survived',axis=1),['Embarked','Title','Sex']).set_index('PassengerId')
dt = DecisionTreeClassifier()
scores = cross_val_score(dt, x, tt.Survived, cv=10)
scores.mean()

0.782342810123709

In [103]:
from sklearn.linear_model import LogisticRegression
for c in [0.001, 0.01, 0.1, 1, 10, 100, 1000]:
    lr = LogisticRegression(C=c)
    scores = cross_val_score(lr, x, tt.Survived, cv=10)
    print(c,scores.mean())



0.001 0.6903413346952674
0.01 0.7913684598796958
0.1 0.8081230847803882
1 0.8294098286233117
10 0.8260892634207242
100 0.8272256270570877
1000 0.8272256270570877


In [98]:
from sklearn.ensemble import RandomForestClassifier
rt= RandomForestClassifier()
scores = cross_val_score(rt, x, tt.Survived, cv=10)
scores.mean()

0.8015577119509704

In [100]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }
clf = GridSearchCV(LogisticRegression(penalty='l2'), param_grid)
GridSearchCV(cv=None,
             estimator=LogisticRegression(C=1.0, intercept_scaling=1,   
               dual=False, fit_intercept=True, penalty='l2', tol=0.0001),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]})

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)


In [145]:
lr = LogisticRegression(C=1)
scores = cross_val_score(lr, x.drop(['Sex_Master'],axis=1), tt.Survived, cv=10)
print(scores.mean())

KeyError: "labels ['Sex_Master'] not contained in axis"

In [146]:
x.head()

,Pclass,Age,SibSp,Parch,Fare,Embarked_female,Embarked_male,Title_C,Title_Q,Title_S,Sex_ Dr,Sex_ Master,Sex_ Miss,Sex_ Mr,Sex_ Mrs
PassengerId,,,,,,,,,,,,,,,
1,3,22.0,1,0,7.2500,0,1,0,0,1,0,0,0,1,0
2,1,38.0,1,0,71.2833,1,0,1,0,0,0,0,0,0,1
3,3,26.0,0,0,7.9250,1,0,0,0,1,0,0,1,0,0
4,1,35.0,1,0,53.1000,1,0,0,0,1,0,0,0,0,1
5,3,35.0,0,0,8.0500,0,1,0,0,1,0,0,0,1,0


In [108]:
test=pd.read_csv('C:/Users/t_louji/Downloads/test1.csv')

In [109]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [111]:
test.drop(['Cabin','Ticket'],axis=1,inplace=True)

In [112]:
test['Title']=test.Name.str.split(',').str[1].str.split('.').str[0]

In [114]:
test.Title=test.Title.apply(chgn)

In [115]:
test.Title.value_counts()

 Mr        245
 Miss       78
 Mrs        73
 Master     21
 Dr          1
Name: Title, dtype: int64

In [118]:
test["Age"] = test.groupby("Title").Age.transform(lambda x: x.fillna(x.mean()))


In [120]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 10 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Fare           417 non-null float64
Embarked       418 non-null object
Title          418 non-null object
dtypes: float64(2), int64(4), object(4)
memory usage: 32.7+ KB


In [121]:
test["Fare"] = test.groupby("Pclass").Fare.transform(lambda x: x.fillna(x.mean()))


In [122]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 10 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Fare           418 non-null float64
Embarked       418 non-null object
Title          418 non-null object
dtypes: float64(2), int64(4), object(4)
memory usage: 32.7+ KB


In [128]:
test_x=pd.get_dummies(test.drop('Name',axis=1),columns=['Embarked','Title','Sex']).set_index('PassengerId')


model=LogisticRegression(C=1)
model.fit(x,tt.Survived)
model.predict(test_x)

array([1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,

In [136]:
pred=pd.concat([test.PassengerId,pd.DataFrame(model.predict(test_x))],axis=1)

In [137]:
pred.to_csv('submission.csv')